In [102]:
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import time

In [103]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [104]:
url1 = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url1)

In [105]:
html = browser.html
soup = bs(html, 'lxml')
#print(soup.prettify())

In [107]:
#Get the Title and Teaser for each post
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

NASA's InSight Detects First Likely 'Quake' on Mars
While their causes are still unknown, one of three shaking events looks a lot like the quakes detected on the Moon by the Apollo missions.


In [42]:
#Find the Featured Image
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

In [43]:
html = browser.html
soup = bs(html, 'html.parser')

In [15]:
browser.click_link_by_id('full_image')

In [16]:
time.sleep(3)
html = browser.html
soup = bs(html, 'html.parser')

In [19]:
x = soup.find('div', class_="fancybox-inner fancybox-skin fancybox-dark-skin fancybox-dark-skin-open")

In [27]:
featured_image_url = x.find('img')['src']
print(featured_image_url)

/spaceimages/images/mediumsize/PIA00271_ip.jpg


In [98]:
#Mars Weather
url3= 'https://twitter.com/marswxreport?lang=en'
browser.visit(url3)

In [99]:
html = browser.html
soup = bs(html, 'html.parser')

In [100]:
mars_weather = soup.find_all('div', class_='js-tweet-text-container')
for tweet in mars_weather: 
    weather_tweet = tweet.find('p').text
    if 'Sol' and 'pressure' in weather_tweet:
        print(weather_tweet)
        break
    else: 
        pass

InSight sol 143 (2019-04-22) low -98.0ºC (-144.3ºF) high -19.8ºC (-3.7ºF)
winds from the SW at 4.5 m/s (10.1 mph) gusting to 12.1 m/s (27.2 mph)
pressure at 7.40 hPapic.twitter.com/EOazNkJqjD


In [34]:
#Mars Facts
url4 = 'https://space-facts.com/mars/'
browser.visit(url4)

In [35]:
html = browser.html
soup = bs(html, 'html.parser')

In [36]:
tables = pd.read_html(url4)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [38]:
df = tables[0]
df.columns = ['Parameter', 'Facts']
df.head()

,Parameter,Facts
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [39]:
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Parameter</th>\n      <th>Facts</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium B

In [58]:
#Mars Hemispheres
url4 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url4)

In [59]:
html = browser.html
soup = bs(html, 'html.parser')

In [76]:
#Store the name of the moons
titlelist = []
results = soup.find_all('div', class_='description')

for result in results:
    title = result.find('h3')
    title_text = title.text
    titlelist.append(title_text)
#titlelist

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [84]:
#Iterate through all pages
img_url = []
for moon in titlelist:
    #Start at base url
    url4 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url4)
    html = browser.html
    soup = bs(html, 'html.parser')
    try:
        time.sleep(3)
        browser.click_link_by_partial_text(moon)  
        # HTML object
        html = browser.html
        # Parse HTML with Beautiful Soup
        soup = bs(html, 'html.parser')
        moonurl = soup.find('div', class_='wide-image-wrapper').find('a')['href']
        img_url.append(moonurl)
    except:
        print('Scraping Complete')
img_url

['http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']

In [89]:
#Merge the two dictionaries
hemisphere_image_urls = []
keys = ['title', 'img_url']
hemisphere_image_urls = [{key: '' for key in keys} for link in img_url]
hemisphere_image_urls

[{'title': '', 'img_url': ''},
 {'title': '', 'img_url': ''},
 {'title': '', 'img_url': ''},
 {'title': '', 'img_url': ''}]

In [91]:
x = 0
for text in titlelist:
    hemisphere_image_urls[x]['title'] = text
    x += 1

In [92]:
x = 0
for link in img_url:
    hemisphere_image_urls[x]['img_url'] = link
    x += 1

In [93]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]